# Data Exploration

## SII Dataset

In [45]:
# Library imports
# -*- coding: utf-8 -*-
import os
import duckdb
from dotenv import load_dotenv
import datetime
import requests
from time import sleep

In [46]:
# Working directory
# -*- coding: utf-8 -*-

# Get the project directory
project_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Data path
data_path = os.path.join(project_dir,"data")


In [47]:
# Lista SII Contribuyentes por Actividad Económica
sii_act_econ_rut = os.path.join(data_path, "raw", "PUB_NOM_ACTECOS.txt")

# Lista SII Contribuyentes por Razón Social
sii_razon_social_rut = os.path.join(data_path, "raw", "PUB_NOMBRES_PJ.txt")

# Lista SII Contribuyentes por Dirección
sii_direccion_rut = os.path.join(data_path, "raw", "PUB_NOM_DOMICILIO.txt")

# Lista SII Contribuyentes por Sucursal
sii_sucursal_rut = os.path.join(data_path, "raw", "PUB_NOM_SUCURSAL.txt")



In [48]:
# Create a DuckDB connection

con = duckdb.connect()

# Consulta para obtener datos SII de actividades económicas relacionadas con vehículos y transporte

query_with_address = f"""
WITH direcciones AS (
   SELECT *,
         ROW_NUMBER() OVER (
             PARTITION BY RUT, DV 
             ORDER BY FECHA DESC
         ) AS rn
   FROM read_csv_auto('{sii_direccion_rut}', delim='\t', header=TRUE)
),
razones_sociales AS (
   SELECT *,
         ROW_NUMBER() OVER (
             PARTITION BY RUT, DV 
             ORDER BY FECHA_INICIO_VIG DESC
         ) AS rn
   FROM read_csv_auto('{sii_razon_social_rut}', delim='\t', header=TRUE)
)
SELECT
   t.RUT,
   t.DV,
   t."CODIGO ACTIVIDAD" as CODIGO_ACTIVIDAD,
   t."DESC. ACTIVIDAD ECONOMICA" as ACTIVIDAD_ECONOMICA,
   t.FECHA as FECHA_ACTIVIDAD_ECONOMICA,
   t."AFECTA A IVA" as AFECTA_IVA,
   t."CATEGORIA TRIBUTARIA" as CATEGORIA_TRIBUTARIA,
   r.RAZON_SOCIAL,
   r.FECHA_INICIO_VIG as FECHA_INICIO_ACTIVIDADES,
   r.FECHA_TG_VIG as FECHA_TERMINO_GIRO_VIGENTE,
   d.VIGENCIA as VIGENCIA_DIRECCION,
   d.FECHA as FECHA_DIRECCION,
   d.TIPO_DIRECCION,
   d.CALLE,
   d.NUMERO,
   d.CIUDAD,
   d.COMUNA,
   d.REGION
FROM read_csv_auto('{sii_act_econ_rut}', delim='\t', header=TRUE) AS t
LEFT JOIN razones_sociales r
   ON t."RUT" = r."RUT" AND t."DV" = r."DV" AND r.rn = 1
LEFT JOIN direcciones d
   ON t."RUT" = d."RUT" AND t."DV" = d."DV" AND d.rn = 1
WHERE (lower(t."DESC. ACTIVIDAD ECONOMICA") LIKE '%vehiculos%'
   OR lower(t."DESC. ACTIVIDAD ECONOMICA") LIKE '%vehiculo%'
   OR lower(t."DESC. ACTIVIDAD ECONOMICA") LIKE '%transportes%'
   OR lower(t."DESC. ACTIVIDAD ECONOMICA") LIKE '%transporte%')
"""
sii_data = con.execute(query_with_address).fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

# API Mercado Público

In [49]:
# Load environment variables from .env file
load_dotenv()
# Get the API key from environment variables
api_key = os.getenv("API_MERCADO_PUBLICO")

In [ ]:

# Función para generar lista de fechas en formato ddmmaaaa
def generar_fechas(inicio, fin):
    inicio_dt = datetime.datetime.strptime(inicio, "%d-%m-%Y")
    fin_dt = datetime.datetime.strptime(fin, "%d-%m-%Y")
    fechas = []
    while inicio_dt <= fin_dt:
        fechas.append(inicio_dt.strftime("%d%m%Y"))
        inicio_dt += datetime.timedelta(days=1)
    return fechas


In [ ]:

# Composicion RUT completo
sii_data["BASE_RUT"] = sii_data["RUT"].apply(lambda x: f"{x:,}".replace(",", "."))

sii_data["RUT_COMPLETO"] = sii_data["BASE_RUT"] + "-" + sii_data["DV"]


# Función para hacer la consulta a la API
def consultar_api_mp(rut, ticket):
    url = (
        f"https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?"
        f"rutempresaproveedor={rut}&ticket={ticket}"
    )
    try:    
        response = requests.get(url)
        if response.status_code == 200:
            print(url)
            return response.json().get("listaEmpresas")
        else:
            print(url)
            print(f"⚠️ Error {response.status_code}")
            return []
    except Exception as e:
        print(f"❌ Excepción en fecha: {e}")
        return []


In [ ]:
def obtener_codigo_empresa(rut, ticket):
    empresas = consultar_api_mp(rut, ticket)
    if empresas and isinstance(empresas, list) and len(empresas) > 0:
        return empresas[0].get('CodigoEmpresa')
    return None


https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=70.017.820-k&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011


In [55]:

obtener_codigo_empresa("76.775.650-K", api_key)

https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=76.775.650-K&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011


'1038613'

In [52]:
car_sellers = sii_data[((sii_data.CIUDAD == "SANTIAGO") | (sii_data.CIUDAD == "STGO")) & (sii_data.AFECTA_IVA == "S")
         & sii_data.CODIGO_ACTIVIDAD.isin(["465905", "451002"])]


In [53]:

sii_data.ACTIVIDAD_ECONOMICA.unique()
# DETECTCAR LOS QUE TIENEN LA PALABRA VENTA y seleccinar las columnas actividad economica y codigo actividad
sii_data[sii_data.ACTIVIDAD_ECONOMICA.str.contains("venta", case=False, na=False)][
    ["ACTIVIDAD_ECONOMICA", "CODIGO_ACTIVIDAD"]].drop_duplicates()


ACTIVIDAD_ECONOMICA CODIGO_ACTIVIDAD
8          VENTA AL POR MAYOR DE VEHICULOS AUTOMOTORES           451001
21   VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...           451002
24   VENTA DE PARTES, PIEZAS Y ACCESORIOS PARA VEHI...           453000
99   VENTA AL POR MENOR DE COMBUSTIBLES PARA VEHICU...           473000
149  VENTA AL POR MAYOR DE EQUIPO DE TRANSPORTE(EXC...           465905

In [57]:
def obtener_codigo_empresa_con_retraso(row):
    codigo = obtener_codigo_empresa(row["RUT_COMPLETO"], api_key)
    if codigo is not None:
       print(f"Consultando RUT: {row['RUT_COMPLETO']} - Código Empresa: {codigo}")
    sleep(5)  # Espera 1 segundo entre llamadas
    return codigo

car_sellers["CODIGO_EMPRESA_MP"] = car_sellers.apply(obtener_codigo_empresa_con_retraso, axis=1)

https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=76.437.690-0&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
Consultando RUT: 76.437.690-0 - Código Empresa: 1415398
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=76.500.497-7&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
Consultando RUT: 76.500.497-7 - Código Empresa: 1720310
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=76.888.054-9&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=77.325.130-4&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=76.078.985-2&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempre

C:\Users\User\AppData\Local\Temp\ipykernel_488\616094541.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  car_sellers["CODIGO_EMPRESA_MP"] = car_sellers.apply(obtener_codigo_empresa_con_retraso, axis=1)


In [59]:
# Aplicar nuevamente la función solo a los que no tienen código, manteniendo el resto del DataFrame

# Filtrar los que no tienen código de empresa
sin_codigo = car_sellers[car_sellers["CODIGO_EMPRESA_MP"].isna()]

# Aplicar la función solo a esos registros
car_sellers.loc[sin_codigo.index, "CODIGO_EMPRESA_MP"] = sin_codigo.apply(obtener_codigo_empresa_con_retraso, axis=1)

https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=76.888.054-9&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=77.325.130-4&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=76.078.985-2&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=77.349.445-2&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=78.225.920-2&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
https://api.mercadopublico.cl/servicios/v1/Publico/Empresas/BuscarProveedor?rutempresaproveedor=78.507.530-7&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
⚠️ Error 500
http

In [60]:
# Guardar los datos en la carpeta data/proccessed en parquet
output_path = os.path.join(data_path, "processed", "car_sellers.parquet")
car_sellers.to_parquet(output_path, index=False)

In [63]:
# Función para hacer la consulta a la API
def consultar_api_licitaciones(fecha, proveedor, ticket):
    url = (
        f"https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?"
        f"fecha={fecha}&CodigoProveedor={proveedor}&ticket={ticket}"
    )
    try:    
        response = requests.get(url)
        if response.status_code == 200:
            print(url)
            return response.json().get("Listado", [])
        else:
            print(f"⚠️ Error {response.status_code} al consultar {fecha}")
            return []
    except Exception as e:
        print(f"❌ Excepción en fecha {fecha}: {e}")
        return []

In [ ]:
car_sellers[car_sellers.CODIGO_EMPRESA_MP.notna()]

RUT DV CODIGO_ACTIVIDAD  \
21      76437690  0           451002   
167     76500497  7           451002   
675     76084494  2           451002   
678     76022152  K           451002   
686     76026261  7           451002   
...          ... ..              ...   
334997  94879000  9           451002   
335033  96962540  7           465905   
335055  96934200  6           451002   
335276  91815000  5           451002   
335334  99549270  9           451002   

                                      ACTIVIDAD_ECONOMICA  \
21      VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   
167     VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   
675     VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   
678     VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   
686     VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   
...                                                   ...   
334997  VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   
335033  VENTA AL POR MAYOR DE EQUIPO DE TRANSPORTE(EXC...   
335055  VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   
335276  VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   
335334  VENTA AL POR MENOR DE VEHICULOS AUTOMOTORES NU...   

       FECHA_ACTIVIDAD_ECONOMICA AFECTA_IVA  CATEGORIA_TRIBUTARIA  \
21                    2023-09-15          S                     1   
167                   2021-04-17          S                     1   
675                   2010-01-15          S                     1   
678                   2008-06-16          S                     1   
686                   2008-07-23          S                     1   
...                          ...        ...                   ...   
334997                2005-05-25          S                     1   
335033                2001-09-10          S                     1   
335055                2000-11-02          S                     1   
335276                1993-01-01          S                     1   
335334                2004-03-02          S                     1   

                                             RAZON_SOCIAL  \
21      INMOBILIARIA Y MOVIMIENTO DE TIERRAS JERICO LI...   
167               INGENIERÍA JUAN CARLOS MORALES E.I.R.L.   
675                                   COMERCIAL DANTE SPA   
678                 SOCIEDAD AUTOMOTRIZ FRANKCAR LIMITADA   
686     INGENIERIA Y SERVICIOS INTEGRALES LOGISTICOS L...   
...                                                   ...   
334997                                 SANDVIK CHILE S.A.   
335033                                      BLUETALK S.A.   
335055                             KOVACS & RODRIGUEZ S A   
335276                        INVERSIONES 10 DE JULIO S A   
335334                            COMERCIAL FULLMOTOS S A   

       FECHA_INICIO_ACTIVIDADES FECHA_TERMINO_GIRO_VIGENTE  ...  \
21                   2006-01-11                        NaT  ...   
167                  2019-03-07                        NaT  ...   
675                  2010-01-15                        NaT  ...   
678                  2008-06-16                        NaT  ...   
686                  2008-07-23                        NaT  ...   
...                         ...                        ...  ...   
334997               1993-01-01                        NaT  ...   
335033               2001-09-02                        NaT  ...   
335055               2000-11-02                        NaT  ...   
335276               1993-01-01                        NaT  ...   
335334               2004-03-02                        NaT  ...   

       FECHA_DIRECCION TIPO_DIRECCION                    CALLE NUMERO  \
21          2023-09-07      DOMICILIO                P HURTADO    678   
167         2015-04-22      DOMICILIO             LA PRIMAVERA   1479   
675         2010-01-15      DOMICILIO                  ANGAMOS    281   
678         2008-06-16      DOMICILIO                AVIADORES   1367   
686         2014-09-05      DOMICILIO              CONSTANTINO    5

In [65]:
FECHA_INICIO = "16-12-2024"
FECHA_FIN = "15-05-2025"
resultados = []
for CODIGO_PROVEEDOR in car_sellers[car_sellers.CODIGO_EMPRESA_MP.notna()]["CODIGO_EMPRESA_MP"].unique():
    TICKET = api_key  # Usamos el API key como ticket
    print(f"🔑 Usando ticket: {TICKET} para el proveedor: {CODIGO_PROVEEDOR}")
    # Generar fechas a consultar
    print(f"Generando fechas desde {FECHA_INICIO} hasta {FECHA_FIN}")
    for fecha in generar_fechas(FECHA_INICIO, FECHA_FIN):
        print(f"🔍 Consultando fecha: {fecha}")
        licitaciones = consultar_api_licitaciones(fecha, CODIGO_PROVEEDOR, TICKET)
        print(licitaciones)
        for lic in licitaciones:
            resultados.append({
                "CodigoProveedor": CODIGO_PROVEEDOR,
                "CodigoExterno": lic.get("CodigoExterno"),
                "Nombre": lic.get("Nombre"),
                "CodigoEstado": lic.get("CodigoEstado"),
                "FechaCierre": lic.get("FechaCierre"),
                "FechaConsulta": fecha  # Agregamos esta columna
            })
        sleep(2)  # 💤 Esperar para no saturar la API


🔑 Usando ticket: 4BEB30E7-0DE3-43CF-9762-467018BC4011 para el proveedor: 1415398
Generando fechas desde 16-12-2024 hasta 15-05-2025
🔍 Consultando fecha: 16122024
https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?fecha=16122024&CodigoProveedor=1415398&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
[]
🔍 Consultando fecha: 17122024
https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?fecha=17122024&CodigoProveedor=1415398&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
[]
🔍 Consultando fecha: 18122024
https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?fecha=18122024&CodigoProveedor=1415398&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
[]
🔍 Consultando fecha: 19122024
https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?fecha=19122024&CodigoProveedor=1415398&ticket=4BEB30E7-0DE3-43CF-9762-467018BC4011
[]
🔍 Consultando fecha: 20122024
https://api.mercadopublico.cl/servicios/v1/publico/licitaciones.json?fecha=20122024&CodigoProveedo

KeyboardInterrupt: 